# Try 0

In [302]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor
import utils_dev as utils
import numpy as np

In [39]:
import openai
from openai import OpenAI

In [340]:
SIMILARITY_THRESHOLD = 0.65

In [202]:
nlx_sample = pd.read_csv('data/nlx_tx_sample_data_gwu.csv') 

In [210]:
nlx_sample = nlx_sample[['description','job_id']]

In [216]:
nlx_sample = nlx_sample[:3]

In [218]:
nlx_sample

,description,job_id
0,Req ID: 29534BR\n\nPOSITION SUMMARY\n\nThis po...,69322097
1,Enters data using computer applications. Assis...,70014023
2,"Kforce has a client in Austin, Texas (TX) that...",70241308


In [152]:
osn_comp_df = pd.read_csv("data/osn_comp.csv")
osn_ind_df = pd.read_csv("data/osn_ind.csv")
osn_pr_df = pd.read_csv("data/osn_pr.csv")
osn_taxonomy = osn_comp_df
osn_taxonomy = osn_taxonomy._append(osn_ind_df, ignore_index = True)
osn_taxonomy = osn_taxonomy._append(osn_pr_df, ignore_index = True)
osn_taxonomy = osn_taxonomy.reset_index()
osn_taxonomy = osn_taxonomy[['index', 'RSD Name']]

In [130]:
# osn_taxonomy = pd.read_json('data/os-commercial-sample-2.9.1.json', lines = True)

In [35]:
job_df = nlx_sample[:2]
job_df = job_df[['job_id', 'description']]

In [37]:
job_df

,job_id,description
0,69322097,Req ID: 29534BR\n\nPOSITION SUMMARY\n\nThis po...
1,70014023,Enters data using computer applications. Assis...


In [379]:
se = Skill_Extractor()

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [369]:
extracted_skills = pd.DataFrame(columns = ['Research ID', 'Skill Tag', 'Correlation Coefficent'])
for job_index, job_row in job_df.iterrows():
    skills_raw = se.extract_raw(job_row['description'])
    # extraction = pd.concat([pd.DataFrame([[job_row['job_id'],skills]], columns=extraction.columns), extraction], ignore_index=True)
    extracted_skills = pd.concat(se.align_extracted(job_row['job_id'],skills_raw), ignore_index=True)

/opt/anaconda3/lib/python3.11/site-packages/skillNer/utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


ValueError: 3 columns passed, passed data had 4 columns

In [371]:
extraction

,ID,Skill_Name
0,70014023,"[mechanically, position, level, additional, fi..."
1,69322097,"[network management, medicaid, levels, collabo..."


In [391]:
skills = ['network management', 'medicaid', 'levels', 'collaborating', 'business communicating', 'business', 'equal', 'industry', 'additional', 'communities', 'interact', 'CSS', 'new hire', 'phoenix', 'read', 'position', 'modules', 'claims claim', 'human resource', 'medical', 'color', 'hospital', 'claims claim medical', 'receive', 'technical training', 'data services', 'maintenance testing', 'operations', 'consulting', 'personal', 'recruit select', 'disabilities', 'integrity', 'leadership']
id = 69322097
aligned = se.align_extracted(id, skills)

In [393]:
aligned

,0,1,2
0,0.804551,69322097,OSN.931
1,0.743978,69322097,OSN.931
2,0.671660,69322097,OSN.931
3,0.678342,69322097,OSN.931
4,0.715925,69322097,OSN.931
5,0.734006,69322097,OSN.931
6,0.747432,69322097,OSN.931


In [347]:
aligned

[['Network Connectivity Troubleshooting', 0.8045507],
 ['Challenge Solving Collaboration', 0.743978],
 ['Table Partitioning Application', 0.67166007],
 ['Spokespeople Training', 0.6783419],
 ['Identify Data Visibility Requirements', 0.7159246],
 ['Stress Testing Troubleshooting', 0.73400617],
 ['Machinery Operation Training', 0.7474322]]

In [387]:
class Skill_Extractor:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_lg")
        self.ner_extractor = SkillExtractor(self.nlp, SKILL_DB, PhraseMatcher)

        return

    def extract_raw(self, input_text):
        extracted_skills_set = set()
        annotations = None
        try:
            annotations = self.ner_extractor.annotate(input_text)
        except ValueError as e:
            print(f"Skipping example, ValueError encountered: {e}")
        except Exception as e:
            print(f"Skipping example, An unexpected error occurred: {e}")

        for item in annotations['results']['full_matches']:
            extracted_skills_set.add(item['doc_node_value'])

        # get ngram_scored
        for item in annotations['results']['ngram_scored']:
            extracted_skills_set.add(item['doc_node_value'])

        return list(extracted_skills_set)

    def align_extracted(self, id, skills):
        # Initialize an empty set to track previously matched skills
        matched_skills_set = set()
        # Create an empty list for the skills that match
        skill_matches = []
        
        # Iterate through each skill in extracted_skills_list
        for extracted_skill in skills:
            # Check if extracted_skill contains non-whitespace characters
            if extracted_skill.strip():
                # Calculate GloVe embedding for the extracted skill
                extracted_embedding = utils.get_embedding(self.nlp, extracted_skill)

                # Initialize variables to store the best match and its similarity score
                best_match = None
                best_similarity = 0.0

                # Iterate through each keyword in key_series (skills from taxonomy)
                for osn_index, osn_row in osn_taxonomy.iterrows():
                    # Calculate embedding for the keywords/skills from taxonomy
                    key_skill = osn_row['RSD Name']
                    key_embedding = utils.get_embedding(self.nlp, key_skill)

                    # Calculate cosine similarity between extracted skill and keyword skill
                    similarity = utils.cosine_similarity(extracted_embedding, key_embedding)

                    # If the similarity score is above the threshold and the skill is not already matched
                    if similarity >= SIMILARITY_THRESHOLD and key_skill not in matched_skills_set:
                        if similarity > best_similarity:
                            best_similarity = similarity
                            best_match = key_skill

                        # Update the set of previously matched skills
                        matched_skills_set.add(key_skill)

                # If best match was found, add it to the list of matched skills
                if best_match:
                    # row_dict = {'Research ID': id,
                    #            'Skill Tag': 'OSN.'+str(osn_index),
                    #            'Correlation Coefficent': best_similarity}
                    row_dict = {id, 'OSN.'+str(osn_index), best_similarity}
                    skill_matches.append(row_dict)
        skill_matches = pd.DataFrame(skill_matches)
        return skill_matches

In [128]:
# print(osn_taxonomy['_id'][0])
# print('\n', osn_taxonomy['syllabus_probability'][0])
# print('\n', osn_taxonomy['field'][0])
# print('\n', osn_taxonomy['language'][0])
# print('\n', osn_taxonomy['institution'][0])
# print('\n', osn_taxonomy['date'][0])
# print('\n', osn_taxonomy['extracted_sections'][0])
# print('\n', osn_taxonomy['citations'][0])


In [234]:
class Utils2:
    @staticmethod
    def get_embedding(nlp, input_text):
        doc = nlp(input_text)
        if len(doc) == 0:
            return np.zeros(300)  # Return zeros for empty texts
        return np.mean([word.vector for word in doc], axis=0)

In [304]:
nlp = spacy.load("en_core_web_sm")
input_text = 'CSS'
op = utils.get_embedding(nlp, input_text)

In [306]:
op

array([-0.40288043, -0.3100045 , -0.09760042, -0.32624003, -0.7334947 ,
        0.5278594 , -0.5032505 ,  0.45880377, -0.7018071 , -1.5867755 ,
        1.6913638 , -0.13303246,  0.04112548,  0.37841654, -0.288118  ,
        0.21130362, -0.7613904 , -1.0909691 ,  0.86744404,  1.041981  ,
       -0.4742679 , -0.29392534, -0.04223326, -0.9524727 ,  1.0489886 ,
        0.7069536 , -0.09185353,  0.9229921 , -0.52893376, -0.03760073,
       -0.67604744,  0.13665986,  0.9968699 ,  0.18521255,  0.11425901,
       -1.5406814 ,  0.47901282,  0.311581  ,  0.54248273,  0.26544604,
       -1.5283296 ,  0.20111972, -0.34784147,  0.50069904, -0.06415503,
       -1.5238144 , -1.1812279 ,  0.50072193,  0.01118879, -0.04951399,
        1.2858853 ,  0.22489552, -0.21933115, -0.01788104, -0.38919097,
       -0.40893978,  0.9359572 ,  1.1169547 , -0.33587185,  0.07719345,
       -1.0685717 , -0.29378808, -0.08032915, -0.534498  ,  0.6080695 ,
        0.7437323 , -0.8135508 , -0.6152866 ,  1.5552881 , -0.07

In [11]:
# Extract skills from text
input_text = """SANCORP is seeking FTE Level II Data Scientist to support the office of DoD Chief Digital and Artificial Intelligence Office (CDAO) Chief Technology Officer (CTO). CDAO CTO requires support in multiple functional areas to ensure deliverables associated with the CDAO Architecture Council, CTO Federation, and CTO Future Architecture Activities. The mission of the CDAO CTO is to accelerate the DoD's adoption of data, analytics, and AI to improve decision making across all levels of the department. The following are examples of responsibilities:

    Support development of insider threat strategy in support of protecting CDAO technical offerings; balance short-term wins with long-term investments to progressively mature CDAO’s defenses against insider threats.
    Lead coordination of policy and strategy related to insider threats with industry partners and other DoD components.
    Lead exploration of data sources that are relevant to measuring, identifying, and defending against insider threats.
    Provide technical leadership in developing capabilities to detect insider threats among large user communities, leveraging combination of statistical, classical machine learning, and deep learning methods.

Sancorp Consulting LLC shall, in its discretion, modify or adjust the position to meet Sancorp’s changing needs. This job description is not a contract and may be adjusted as deemed appropriate at Sancorp’s sole discretion.

Sancorp Consulting, LLC, is an SDVOSB and SBA 8(a) company seeking highly motivated and qualified professionals and offer an attractive salary and benefits package that includes: Medical, Dental, life and Disability Insurance; 401K, and holidays to ensure the highest quality of life for our employees. Please visit our website for more information at www.sancorpconsulting.com.

Sancorp Consulting, LLC is an equal opportunity employer. At Sancorp Consulting, LLC we are committed to providing equal employment opportunities (EEO) to all employees and applicants without regard to race color, religion, sex, national origin, age, disability, or any other protected characteristic as defined by applicable law. We strive to create an inclusive and diverse workplace where everyone feels valued, respected, and supported."""


In [71]:
import numpy as np

In [35]:
def get_embedding(nlp, input_text):
    doc = nlp(input_text)
    if len(doc) == 0:
        return np.zeros(300)  # Return zeros for empty texts
    return np.mean([word.vector for word in doc], axis=0)

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [150]:
raw_skills = ['python', 'Data warehouses', 'Java']

taxonomy = {
    1: ['ESCO.1', 'Python Programming'], 
    2: ['ESCO.2', 'maintain database performance'], 
    3: ['ESCO.3', 'maintain warehouse database'], 
    4: ['ESCO.4', 'Java Programming'], 
    5: ['ESCO.5', 'JavaScript Framework'],
    6: ['OSN.1', 'Python scripting'],
    7: ['OSN.2', 'Java Enterprise Edition'],
    8: ['OSN.3', 'Relational Databases']
}

In [152]:
tx_df = pd.DataFrame(taxonomy).T
tx_df.columns = ['SkillTag', 'SkillLabel']

In [184]:
skill_matches = []

for raw_skill in raw_skills:
    embd1 = get_embedding(nlp, raw_skill)

    best_match = None
    best_similarity = 0.0
    matched_skill_set = set()
    
    for index, row in tx_df.iterrows():
        tag = row['SkillTag']
        label = row['SkillLabel']
        embd2 = get_embedding(nlp, label)
        similarity = cosine_similarity(embd1, embd2) 
        print(f'{raw_skill} X {tag}({label}):\t{similarity}')
        # print(raw_skill, 'X', tag, ': ', label, '\t-\t', similarity)
        
        if similarity > 0.65 and tag not in matched_skill_set:
            if similarity > best_similarity:
                best_match = tag
                best_similarity = similarity
                matched_skill_set.add(tag)
                

    if best_match:
        skill_matches.append({
            "RawSkill": raw_skill,
            "SkillTag": best_match,
            "corr_coeff": best_similarity
        })

python X ESCO.1(Python Programming):	0.6568669676780701
python X ESCO.2(maintain database performance):	0.22788506746292114
python X ESCO.3(maintain warehouse database):	0.26050493121147156
python X ESCO.4(Java Programming):	0.5509223341941833
python X ESCO.5(JavaScript Framework):	0.4448027014732361
python X OSN.1(Python scripting):	0.7103729844093323
python X OSN.2(Java Enterprise Edition):	0.3846420347690582
python X OSN.3(Relational Databases):	0.1756518930196762
Data warehouses X ESCO.1(Python Programming):	0.474836140871048
Data warehouses X ESCO.2(maintain database performance):	0.48530834913253784
Data warehouses X ESCO.3(maintain warehouse database):	0.601143479347229
Data warehouses X ESCO.4(Java Programming):	0.4844682216644287
Data warehouses X ESCO.5(JavaScript Framework):	0.5272994041442871
Data warehouses X OSN.1(Python scripting):	0.3802958130836487
Data warehouses X OSN.2(Java Enterprise Edition):	0.5426174998283386
Data warehouses X OSN.3(Relational Databases):	0.6523

In [186]:
skill_matches

[{'RawSkill': 'python', 'SkillTag': 'OSN.1', 'corr_coeff': 0.710373},
 {'RawSkill': 'Data warehouses', 'SkillTag': 'OSN.3', 'corr_coeff': 0.6523371},
 {'RawSkill': 'Java', 'SkillTag': 'ESCO.4', 'corr_coeff': 0.9181258}]

# Try 1

In [3]:
import openai
from openai import OpenAI
import pandas as pd
import spacy

In [5]:
from params import DATA_PATH, SKILL_TAXONOMY, SIMILARITY_THRESHOLD, AI_MODEL_ID, API_KEY
import utils

In [7]:
from Skill_Extractor_dev import Skill_Extractor

In [9]:
se = Skill_Extractor()

In [15]:
raw_skills = se.__extract_raw(input_text)

AttributeError: 'Skill_Extractor' object has no attribute '__extract_raw'

In [190]:
raw_skills = ['Investing', 	'Integrity', 	'Diversity', 	'Strategic thinking', 	'Value creation', 	'Client experience', 	'Employee development', 	'Optimism', 	'Results-oriented', 	'Curiosity', 	'Innovation', 	'Adaptability', 	'Data warehousing', 	'Data analysis', 	'Data modeling', 	'SQL', 	'Star schemas', 	'Snowflake schemas', 	'Data profiling', 	'Data architecture', 	'Dimensional modeling', 	'Business requirements', 	'Data insights', 	'Kimball modeling', 	'Inmon concepts', 	'Database technologies', 	'Erwin', 	'Teradata', 	'Big Data', 	'Hadoop', 	'Reporting tools', 	'Business Objects', 	'OBIEE', 	'Cognos', 	'ETL methodologies', 	'Informatica', 	'Datastage', 	'Consensus building', 	'Interpersonal skills', 	'Analytical skills', 	'Problem-solving', 	'Influencing', 	'Prioritization', 	'Decision-making', 	'Conflict resolution', 	'Project management', 	'Communication skills', 	'MS Project', 	'Excel', 	'MS Word', 	'Visio', 	'Outlook', 	'Wellness programs', 	'401k', 	'Employee discounts', 	'Tuition reimbursement', 	'Diversity initiatives']

In [188]:
osn_comp_df = pd.read_csv("data/osn_comp.csv")
osn_ind_df = pd.read_csv("data/osn_ind.csv")
osn_pr_df = pd.read_csv("data/osn_pr.csv")
osn_taxonomy = osn_comp_df
osn_taxonomy = osn_taxonomy._append(osn_ind_df, ignore_index = True)
osn_taxonomy = osn_taxonomy._append(osn_pr_df, ignore_index = True)
osn_taxonomy = osn_taxonomy.reset_index()
osn_taxonomy = osn_taxonomy[['index', 'RSD Name']]

In [192]:
esco_digital_df = pd.read_csv("data/ESCO dataset - v1.2.0 - classification - en - csv/digitalSkillsCollection_en.csv")

In [194]:
osn_comp_df = osn_comp_df.reset_index()
osn_comp_df = osn_comp_df[['index', 'RSD Name']]
esco_digital_df = esco_digital_df.reset_index()
esco_digital_df = esco_digital_df[['index', 'preferredLabel']]

In [196]:
taxonomy = pd.read_csv('data/combined.csv')

In [37]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [198]:
skill_matches = []

for raw_skill in raw_skills:
    embd1 = get_embedding(nlp, raw_skill)

    best_match = None
    best_similarity = 0.0
    matched_skill_set = set()
    
    for index, row in tx_df.iterrows():
        tag = row['SkillTag']
        label = row['SkillLabel']
        embd2 = get_embedding(nlp, label)
        similarity = cosine_similarity(embd1, embd2) 
        # print(f'{raw_skill} X {tag}({label}):\t{similarity}')
        
        if similarity > 0.65 and tag not in matched_skill_set:
            if similarity > best_similarity:
                best_match = tag
                best_similarity = similarity
                matched_skill_set.add(tag)
                

    if best_match:
        skill_matches.append({
            "RawSkill": raw_skill,
            "SkillTag": best_match,
            "corr_coeff": best_similarity
        })    

Investing X ESCO.1(Python Programming):	0.44457077980041504
Investing X ESCO.2(maintain database performance):	0.5615065693855286
Investing X ESCO.3(maintain warehouse database):	0.49951857328414917
Investing X ESCO.4(Java Programming):	0.4282401502132416
Investing X ESCO.5(JavaScript Framework):	0.49301525950431824
Investing X OSN.1(Python scripting):	0.36667343974113464
Investing X OSN.2(Java Enterprise Edition):	0.5572608709335327
Investing X OSN.3(Relational Databases):	0.7119366526603699
Integrity X ESCO.1(Python Programming):	0.3986188471317291
Integrity X ESCO.2(maintain database performance):	0.499983549118042
Integrity X ESCO.3(maintain warehouse database):	0.39806440472602844
Integrity X ESCO.4(Java Programming):	0.382937490940094
Integrity X ESCO.5(JavaScript Framework):	0.5103845596313477
Integrity X OSN.1(Python scripting):	0.23724667727947235
Integrity X OSN.2(Java Enterprise Edition):	0.4982088506221771
Integrity X OSN.3(Relational Databases):	0.6754821538925171
Diversit

/var/folders/n8/b6_h9cbd1r3fjkr_2zjs39qw0000gn/T/ipykernel_58367/27609111.py:8: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


Conflict resolution X OSN.1(Python scripting):	0.34037089347839355
Conflict resolution X OSN.2(Java Enterprise Edition):	0.5371614098548889
Conflict resolution X OSN.3(Relational Databases):	0.6657276153564453
Project management X ESCO.1(Python Programming):	0.5485318899154663
Project management X ESCO.2(maintain database performance):	0.6406599283218384
Project management X ESCO.3(maintain warehouse database):	0.5659531950950623
Project management X ESCO.4(Java Programming):	0.5330867767333984
Project management X ESCO.5(JavaScript Framework):	0.6025853753089905
Project management X OSN.1(Python scripting):	0.4322394132614136
Project management X OSN.2(Java Enterprise Edition):	0.6364972591400146
Project management X OSN.3(Relational Databases):	0.7242257595062256
Communication skills X ESCO.1(Python Programming):	0.5214516520500183
Communication skills X ESCO.2(maintain database performance):	0.6398324966430664
Communication skills X ESCO.3(maintain warehouse database):	0.49212190508

In [200]:
skill_matches

[{'RawSkill': 'Investing', 'SkillTag': 'OSN.3', 'corr_coeff': 0.71193665},
 {'RawSkill': 'Integrity', 'SkillTag': 'OSN.3', 'corr_coeff': 0.67548215},
 {'RawSkill': 'Value creation', 'SkillTag': 'OSN.3', 'corr_coeff': 0.738745},
 {'RawSkill': 'Client experience',
  'SkillTag': 'ESCO.2',
  'corr_coeff': 0.70192987},
 {'RawSkill': 'Employee development',
  'SkillTag': 'OSN.3',
  'corr_coeff': 0.7229016},
 {'RawSkill': 'Innovation', 'SkillTag': 'OSN.3', 'corr_coeff': 0.7449121},
 {'RawSkill': 'Adaptability', 'SkillTag': 'ESCO.2', 'corr_coeff': 0.73178256},
 {'RawSkill': 'Data warehousing',
  'SkillTag': 'OSN.3',
  'corr_coeff': 0.69638705},
 {'RawSkill': 'Data analysis', 'SkillTag': 'OSN.3', 'corr_coeff': 0.73029923},
 {'RawSkill': 'Data modeling', 'SkillTag': 'OSN.3', 'corr_coeff': 0.741107},
 {'RawSkill': 'SQL', 'SkillTag': 'OSN.1', 'corr_coeff': 0.67050135},
 {'RawSkill': 'Data profiling', 'SkillTag': 'OSN.3', 'corr_coeff': 0.7301169},
 {'RawSkill': 'Data architecture',
  'SkillTag': 'O

In [220]:
extracted =[[{'Research ID': 69322097, 'Raw Skill': 'maintenance testing', 'Skill Tag': 'ESCO.1278', 'Correlation Coefficient': 0.88474965}, {'Research ID': 69322097, 'Raw Skill': 'industry', 'Skill Tag': 'ESCO.930', 'Correlation Coefficient': 0.8775265}, {'Research ID': 69322097, 'Raw Skill': 'business communicating', 'Skill Tag': 'ESCO.113', 'Correlation Coefficient': 0.91648597}, {'Research ID': 69322097, 'Raw Skill': 'operations', 'Skill Tag': 'ESCO.805', 'Correlation Coefficient': 0.8502768}, {'Research ID': 69322097, 'Raw Skill': 'receive', 'Skill Tag': 'ESCO.71', 'Correlation Coefficient': 0.5794006}, {'Research ID': 69322097, 'Raw Skill': 'consulting', 'Skill Tag': 'ESCO.187', 'Correlation Coefficient': 0.81846404}, {'Research ID': 69322097, 'Raw Skill': 'CSS', 'Skill Tag': 'ESCO.1150', 'Correlation Coefficient': 1.0}, {'Research ID': 69322097, 'Raw Skill': 'leadership', 'Skill Tag': 'ESCO.856', 'Correlation Coefficient': 0.6670284}, {'Research ID': 69322097, 'Raw Skill': 'claims claim medical', 'Skill Tag': 'ESCO.1075', 'Correlation Coefficient': 0.69185805}, {'Research ID': 69322097, 'Raw Skill': 'medical', 'Skill Tag': 'ESCO.892', 'Correlation Coefficient': 0.9245661}, {'Research ID': 69322097, 'Raw Skill': 'personal', 'Skill Tag': 'ESCO.250', 'Correlation Coefficient': 0.76929283}, {'Research ID': 69322097, 'Raw Skill': 'modules', 'Skill Tag': 'ESCO.365', 'Correlation Coefficient': 0.7899949}, {'Research ID': 69322097, 'Raw Skill': 'recruit select', 'Skill Tag': 'ESCO.487', 'Correlation Coefficient': 0.6540943}, {'Research ID': 69322097, 'Raw Skill': 'network management', 'Skill Tag': 'ESCO.57', 'Correlation Coefficient': 0.91445017}, {'Research ID': 69322097, 'Raw Skill': 'phoenix', 'Skill Tag': 'ESCO.129', 'Correlation Coefficient': 0.24092473}, {'Research ID': 69322097, 'Raw Skill': 'hospital', 'Skill Tag': 'ESCO.892', 'Correlation Coefficient': 0.5352579}, {'Research ID': 69322097, 'Raw Skill': 'integrity', 'Skill Tag': 'ESCO.92', 'Correlation Coefficient': 0.74480486}, {'Research ID': 69322097, 'Raw Skill': 'read', 'Skill Tag': 'ESCO.130', 'Correlation Coefficient': 0.5076869}, {'Research ID': 69322097, 'Raw Skill': 'color', 'Skill Tag': 'ESCO.470', 'Correlation Coefficient': 0.6941998}, {'Research ID': 69322097, 'Raw Skill': 'collaborating', 'Skill Tag': 'OSN.120', 'Correlation Coefficient': 0.83662075}, {'Research ID': 69322097, 'Raw Skill': 'levels', 'Skill Tag': 'ESCO.674', 'Correlation Coefficient': 0.6362278}, {'Research ID': 69322097, 'Raw Skill': 'human resource', 'Skill Tag': 'ESCO.1284', 'Correlation Coefficient': 0.7435226}, {'Research ID': 69322097, 'Raw Skill': 'claims claim', 'Skill Tag': 'ESCO.602', 'Correlation Coefficient': 0.5618553}, {'Research ID': 69322097, 'Raw Skill': 'business', 'Skill Tag': 'ESCO.761', 'Correlation Coefficient': 0.8897122}, {'Research ID': 69322097, 'Raw Skill': 'position', 'Skill Tag': 'ESCO.517', 'Correlation Coefficient': 0.6855561}, {'Research ID': 69322097, 'Raw Skill': 'technical training', 'Skill Tag': 'ESCO.311', 'Correlation Coefficient': 0.83091843}, {'Research ID': 69322097, 'Raw Skill': 'interact', 'Skill Tag': 'ESCO.801', 'Correlation Coefficient': 0.7778724}, {'Research ID': 69322097, 'Raw Skill': 'disabilities', 'Skill Tag': 'ESCO.945', 'Correlation Coefficient': 0.732762}, {'Research ID': 69322097, 'Raw Skill': 'additional', 'Skill Tag': 'ESCO.775', 'Correlation Coefficient': 0.77065897}, {'Research ID': 69322097, 'Raw Skill': 'equal', 'Skill Tag': 'ESCO.296', 'Correlation Coefficient': 0.43665254}, {'Research ID': 69322097, 'Raw Skill': 'new hire', 'Skill Tag': 'ESCO.548', 'Correlation Coefficient': 0.6886523}, {'Research ID': 69322097, 'Raw Skill': 'data services', 'Skill Tag': 'ESCO.766', 'Correlation Coefficient': 0.93508047}, {'Research ID': 69322097, 'Raw Skill': 'communities', 'Skill Tag': 'ESCO.330', 'Correlation Coefficient': 0.7319974}, {'Research ID': 69322097, 'Raw Skill': 'medicaid', 'Skill Tag': 'ESCO.892', 'Correlation Coefficient': 0.5985519}], [{'Research ID': 70014023, 'Raw Skill': 'additional', 'Skill Tag': 'ESCO.775', 'Correlation Coefficient': 0.77065897}, {'Research ID': 70014023, 'Raw Skill': 'mechanically', 'Skill Tag': 'ESCO.614', 'Correlation Coefficient': 0.79304266}, {'Research ID': 70014023, 'Raw Skill': 'position', 'Skill Tag': 'ESCO.517', 'Correlation Coefficient': 0.6855561}, {'Research ID': 70014023, 'Raw Skill': 'level', 'Skill Tag': 'ESCO.907', 'Correlation Coefficient': 0.59792906}, {'Research ID': 70014023, 'Raw Skill': 'filing', 'Skill Tag': 'ESCO.292', 'Correlation Coefficient': 0.60839117}, {'Research ID': 70014023, 'Raw Skill': 'scheduling', 'Skill Tag': 'ESCO.1121', 'Correlation Coefficient': 0.7256151}], [{'Research ID': 70241308, 'Raw Skill': 'sources', 'Skill Tag': 'ESCO.487', 'Correlation Coefficient': 0.68492126}, {'Research ID': 70241308, 'Raw Skill': 'consolidating', 'Skill Tag': 'ESCO.775', 'Correlation Coefficient': 0.81346995}, {'Research ID': 70241308, 'Raw Skill': 'financial data', 'Skill Tag': 'ESCO.433', 'Correlation Coefficient': 0.9325687}, {'Research ID': 70241308, 'Raw Skill': 'systems logic', 'Skill Tag': 'ESCO.1025', 'Correlation Coefficient': 0.90447015}, {'Research ID': 70241308, 'Raw Skill': 'data migration', 'Skill Tag': 'ESCO.816', 'Correlation Coefficient': 0.9455265}, {'Research ID': 70241308, 'Raw Skill': 'data analysis', 'Skill Tag': 'ESCO.757', 'Correlation Coefficient': 0.9671367}, {'Research ID': 70241308, 'Raw Skill': 'sql', 'Skill Tag': 'ESCO.443', 'Correlation Coefficient': 0.77139634}, {'Research ID': 70241308, 'Raw Skill': 'sql databases', 'Skill Tag': 'ESCO.322', 'Correlation Coefficient': 0.91066796}, {'Research ID': 70241308, 'Raw Skill': 'financial analyst', 'Skill Tag': 'ESCO.761', 'Correlation Coefficient': 0.6976437}, {'Research ID': 70241308, 'Raw Skill': 'sql server', 'Skill Tag': 'ESCO.421', 'Correlation Coefficient': 0.8271698}, {'Research ID': 70241308, 'Raw Skill': 'tools', 'Skill Tag': 'ESCO.792', 'Correlation Coefficient': 0.84649366}, {'Research ID': 70241308, 'Raw Skill': 'access', 'Skill Tag': 'ESCO.417', 'Correlation Coefficient': 0.8446417}, {'Research ID': 70241308, 'Raw Skill': 'unstructured data', 'Skill Tag': 'ESCO.991', 'Correlation Coefficient': 1.0}, {'Research ID': 70241308, 'Raw Skill': 'revenue recognition', 'Skill Tag': 'ESCO.159', 'Correlation Coefficient': 0.73932314}, {'Research ID': 70241308, 'Raw Skill': 'financial service', 'Skill Tag': 'ESCO.187', 'Correlation Coefficient': 0.84712994}]]


In [228]:
pd.DataFrame(extracted)

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,"{'Research ID': 69322097, 'Raw Skill': 'mainte...","{'Research ID': 69322097, 'Raw Skill': 'indust...","{'Research ID': 69322097, 'Raw Skill': 'busine...","{'Research ID': 69322097, 'Raw Skill': 'operat...","{'Research ID': 69322097, 'Raw Skill': 'receiv...","{'Research ID': 69322097, 'Raw Skill': 'consul...","{'Research ID': 69322097, 'Raw Skill': 'CSS', ...","{'Research ID': 69322097, 'Raw Skill': 'leader...","{'Research ID': 69322097, 'Raw Skill': 'claims...","{'Research ID': 69322097, 'Raw Skill': 'medica...",...,"{'Research ID': 69322097, 'Raw Skill': 'positi...","{'Research ID': 69322097, 'Raw Skill': 'techni...","{'Research ID': 69322097, 'Raw Skill': 'intera...","{'Research ID': 69322097, 'Raw Skill': 'disabi...","{'Research ID': 69322097, 'Raw Skill': 'additi...","{'Research ID': 69322097, 'Raw Skill': 'equal'...","{'Research ID': 69322097, 'Raw Skill': 'new hi...","{'Research ID': 69322097, 'Raw Skill': 'data s...","{'Research ID': 69322097, 'Raw Skill': 'commun...","{'Research ID': 69322097, 'Raw Skill': 'medica..."
1,"{'Research ID': 70014023, 'Raw Skill': 'additi...","{'Research ID': 70014023, 'Raw Skill': 'mechan...","{'Research ID': 70014023, 'Raw Skill': 'positi...","{'Research ID': 70014023, 'Raw Skill': 'level'...","{'Research ID': 70014023, 'Raw Skill': 'filing...","{'Research ID': 70014023, 'Raw Skill': 'schedu...",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"{'Research ID': 70241308, 'Raw Skill': 'source...","{'Research ID': 70241308, 'Raw Skill': 'consol...","{'Research ID': 70241308, 'Raw Skill': 'financ...","{'Research ID': 70241308, 'Raw Skill': 'system...","{'Research ID': 70241308, 'Raw Skill': 'data m...","{'Research ID': 70241308, 'Raw Skill': 'data a...","{'Research ID': 70241308, 'Raw Skill': 'sql', ...","{'Research ID': 70241308, 'Raw Skill': 'sql da...","{'Research ID': 70241308, 'Raw Skill': 'financ...","{'Research ID': 70241308, 'Raw Skill': 'sql se...",...,None,None,None,None,None,None,None,None,None,None


# Try 2

In [42]:
import pandas as pd
import spacy
import numpy as np

In [46]:
nlp = spacy.load("en_core_web_lg")

In [54]:
SIMILARITY_THRESHOLD = 0.65

In [92]:
def get_embedding(nlp, input_text):
    doc = nlp(input_text)
    if len(doc) == 0:
        return np.zeros(300)  # Return zeros for empty texts
    return np.mean([word.vector for word in doc], axis=0)

def cosine_similarity(vec1, vec2):
    product_of_magnitude = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    if product_of_magnitude == 0.0:
        return 0.0
    return np.dot(vec1, vec2) / (product_of_magnitude)

# skill_db_df = pd.read_csv('/Users/vedantmahangade/Projects/PSCWP/LAiSER/laiser/input/combined.csv')
# for index, row in skill_db_df.iterrows():
#     tag = row['SkillTag']
#     label = row['SkillLabel']
#     db_skill_embedding = get_embedding(nlp, label)
#     if np.linalg.norm(db_skill_embedding) == 0.0:
#         print(index, tag, label)

In [85]:
def align_skills(raw_skills, document_id='0'):
   
    # dataframe for skill taxonomy database
    skill_db_df = pd.read_csv('/Users/vedantmahangade/Projects/PSCWP/LAiSER/laiser/input/combined.csv')

    skill_matches = pd.DataFrame(columns = ['Research ID', 'Raw Skill', 'Skill Tag', 'Correlation Coefficient'])

    # iterate over extracted skills
    for raw_skill in raw_skills:

        # get vectorized embedding for raw skill
        raw_skill_embedding = get_embedding(nlp, raw_skill)

        best_match = None
        best_similarity = 0.0
        matched_skill_set = set()

        # iterate over each row in skill taxonomy db
        for index, row in skill_db_df.iterrows():
            tag = row['SkillTag']
            label = row['SkillLabel']

            # get vectorized embedding for skill in taxonomy db
            db_skill_embedding = get_embedding(nlp, label)

            # get cosine similarity between raw skill and skill from taxonomy db
            similarity = cosine_similarity(raw_skill_embedding, db_skill_embedding)
            # print(f'{raw_skill} X {tag}({label}):\t{similarity}')

            # if cosine similarity > threshold and not already added then update the best tag
            if similarity > SIMILARITY_THRESHOLD and tag not in matched_skill_set:
                if similarity > best_similarity:
                    best_match = tag
                    best_similarity = similarity
                    matched_skill_set.add(tag)

        # add the best similarity and best match to the output
        if best_match:
            temp = {
                "Research ID": document_id,
                "Raw Skill": raw_skill,
                "Skill Tag": best_match,
                "Correlation Coefficient": best_similarity
            }
            skill_matches = skill_matches._append(temp, ignore_index = True)
            # skill_matches.append({
            #     "Research ID": document_id,
            #     "Raw Skill": raw_skill,
            #     "Skill Tag": best_match,
            #     "Correlation Coefficient": best_similarity
            # })

    return skill_matches

def extractor(data, id_column='Research ID', text_column='Text'):
    extracted = pd.DataFrame(columns = ['Research ID', 'Raw Skill', 'Skill Tag', 'Correlation Coefficient'])
    for index, row in data.iterrows():
        research_id = row[id_column]
        input_text = row[text_column]
        raw_skills = extract_raw(input_text)
        aligned_skills = align_skills(raw_skills, research_id)
        extracted = extracted._append(aligned_skills, ignore_index = True)

In [50]:
raw_skills_list=[['additional', 'mechanically', 'position', 'level', 'filing', 'scheduling'],
 [' sources', 'consolidating', 'financial data', 'systems logic', 'data migration', 'data analysis', 'sql', 'sql databases', 'financial analyst', 'sql server', 'tools', 'access', 'unstructured data', 'revenue recognition', 'financial service']]

In [99]:
i=1
extracted = pd.DataFrame(columns = ['Research ID', 'Raw Skill', 'Skill Tag', 'Correlation Coefficient'])
for raw_skills in raw_skills_list:
    research_id = i
    aligned_skills = align_skills(raw_skills, research_id)
    extracted = extracted._append(aligned_skills, ignore_index = True)

/var/folders/n8/b6_h9cbd1r3fjkr_2zjs39qw0000gn/T/ipykernel_71004/535841310.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  skill_matches = skill_matches._append(temp, ignore_index = True)
/var/folders/n8/b6_h9cbd1r3fjkr_2zjs39qw0000gn/T/ipykernel_71004/2503964264.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted = extracted._append(aligned_skills, ignore_index = True)
/var/folders/n8/b6_h9cbd1r3fjkr_2zjs39qw0000gn/T/ipykernel_71004/535841310.py:45: FutureWarning: The behavior of

In [101]:
extracted

,Research ID,Raw Skill,Skill Tag,Correlation Coefficient
0,1,additional,ESCO.775,0.770659
1,1,mechanically,ESCO.614,0.793043
2,1,position,ESCO.517,0.685556
3,1,scheduling,ESCO.1121,0.725615
4,1,sources,ESCO.487,0.684921
5,1,consolidating,ESCO.775,0.813470
6,1,financial data,ESCO.433,0.932569
7,1,systems logic,ESCO.1025,0.904470
8,1,data migration,ESCO.816,0.945526
9,1,data analysis,ESCO.757,0.967137


# Extraction Function Execution

In [116]:
import pandas as pd
# Add the project root directory to the Python path
import sys
import os

# Assuming you are running this from the notebooks directory
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

# Now you can import the skill_extractor module
from laiser.skill_extractor import SkillExtractor
from laiser.params import INPUT_PATH

In [128]:
from laiser.skill_extractor import SkillExtractor

In [118]:
nlx_sample = pd.read_csv(os.path.join(INPUT_PATH, 'nlx_tx_sample_data_gwu.csv'))

nlx_sample = nlx_sample[['description', 'job_id']]
nlx_sample = nlx_sample[1:3]

In [120]:
nlx_sample

,description,job_id
1,Enters data using computer applications. Assis...,70014023
2,"Kforce has a client in Austin, Texas (TX) that...",70241308


In [130]:
SkillExtractor

skillNer.skill_extractor_class.SkillExtractor